# Sampling Notebook
---
## Setup imports and initial comment retrieval

In [2]:
import pymongo
import random
import pandas as pd
import configparser
from pathlib import Path
from typing import Any
import re
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup

# check average length of content
# check min and max lengths of content 

config = configparser.RawConfigParser()
config.read(Path("../application.properties"))

def get_mongoclient(config: configparser.RawConfigParser) -> pymongo.MongoClient:

    user = config.get('DatabaseSection', 'mongodb.user')
    password = config.get('DatabaseSection', 'mongodb.password')
    hostname = config.get('DatabaseSection', 'mongodb.hostname')
    port = config.get('DatabaseSection', 'mongodb.port')
    options = config.get('DatabaseSection', 'mongodb.options')

    conn_str = f"mongodb://{user}:{password}@{hostname}:{port}/{options}"

    return pymongo.MongoClient(conn_str, serverSelectionTimeoutMS=5000, unicode_decode_error_handler='ignore')

comments_db = config.get('DatabaseSection', 'mongodb.database.comments')
db = get_mongoclient(config=config)[comments_db]
# pattern = re.compile('\.java$', re.IGNORECASE)
comments = list(db.comments_only_java.find({'filtered': False}))


In [3]:
len(comments)

895423

## Helper functions and Stratified comment sample setup

In [4]:
from typing import Callable, Dict, List
from bs4 import Tag
from selenium.common.exceptions import TimeoutException

def sample_and_persist_df(df: pd.DataFrame, sample_size: int, path: Path) -> pd.DataFrame:

    seed = 24104 
    if sample_size == 1:
        sample_df = df.sample(frac=1, random_state=seed).reset_index(drop = True)
    else:
        sample_df = df.sample(sample_size, random_state=seed)


    sample_df.to_excel(path, index=False)

    return sample_df


def create_context_df(df_to_filter_path: Path, filter_df_path: Path, content_col_name: str, context_label: str) -> pd.DataFrame:

    # TODO dont need 2 dfs! can just use 1 df and filter that one based on label!!

    df_to_filter = pd.read_excel(df_to_filter_path)
    filter_df = pd.read_excel(filter_df_path, sheet_name='Comments Labeled',
                                        usecols=content_col_name,
                                            na_values='',
                                            nrows=len(df_to_filter), ).fillna('nan')
                                            
    df_to_filter.reset_index(drop=True, inplace=True)
    
    context_df = df_to_filter[filter_df.iloc[:,0].str.contains(context_label)]

    # generate comment context urls
    context_df['COMM_COMMIT_URL'] = get_commit_diff_comment_urls(
        context_df['COMM_COMMIT_URL'].values,
        context_df['FILE_PATH'].values,
        context_df['HUNK_NEW_LINE'].values)
    
    return context_df

def get_commit_diff_comment_urls(commit_urls: List[str], file_paths: List[str], line_numbs: List[int]) -> List[str]:
    # Set the string to search for within the diff
    comment_urls = []

    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
    
    for idx, commit_url in enumerate(commit_urls):
        html = ''
        # Construct the GitHub API URL
        driver.get(commit_url)

        wait = WebDriverWait(driver, 20)
        try:
            end_of_file_path = "/" + file_paths[idx].split("/")[-1]
            element = wait.until(EC.presence_of_element_located((By.PARTIAL_LINK_TEXT, end_of_file_path))) 
            html = driver.page_source

            soup = BeautifulSoup(html, "html.parser")
            diff_anchor = soup.find(filter_links(end_of_file_path))
            if diff_anchor:
                commit_url = f'{commit_url}{diff_anchor["href"]}R{line_numbs[idx]}'
            # TODO activate this else as last resort 
            # can actually be the case that file path can be found but then commit seems to be wrong so then need to have fail safe to add blob path
            # example: https://github.com/apache/derby/commit/f16c46cbdd5be8dd9bdcee935ec1f68970146478 and ending file path: /OperatorNode.java
            # with comment: "No query. The SqlXmlUtil instance is still on the stack. Pop it to restore the initial state of the stack."
            # else:
            #     commit_url = f'{commit_url.replace("commit", "blob")}/{file_paths[idx]}#L{line_numbs[idx]}'

        except TimeoutException:
            commit_url = f'{commit_url.replace("commit", "blob")}/{file_paths[idx]}#L{line_numbs[idx]}'

        comment_urls.append(commit_url)

    driver.quit()

    return comment_urls

def filter_links(filter_text: str) -> Callable:
    
    def filter_links_ending_with_str(tag: Tag):
        return tag.name == 'a' and tag.text.endswith(filter_text)
    
    return filter_links_ending_with_str

def stratify_prop_to_project_name(df: pd.DataFrame, sample_frac: float) -> pd.DataFrame:
   random_seed_strat = 2802234
   df['PROJECT'].unique()[0]

#    return df.sample(max(1, int(sample_frac*len(df))), random_state=random_seed_strat)
   return df.sample(3, random_state=random_seed_strat)

def get_rand_item(comments: List[Dict]) -> Dict:
    random.seed(26225)
    return random.sample(comments, 1)

def get_comment_df(comments: Dict) -> pd.DataFrame:
    ids = [comment_dict['_id'] for comment_dict in comments]
    contents = [comment_dict['content'] for comment_dict in comments]
    project_names = [comment_dict['project_name'] for comment_dict in comments]
    comm_types = [comment_dict['type'] for comment_dict in comments]
    comm_dates = [comment_dict['committer_date'] for comment_dict in comments]
    comm_file_path = [comment_dict['file_path'] for comment_dict in comments]
    comm_vcs = ['{}/commit/{}'.format(comment_dict['vcs_url'].replace('.git', ''),
                                       comment_dict['commit_hash']) for comment_dict in comments]
    comm_hunk_old_line = [comment_dict['hunk_old_start'] for comment_dict in comments]
    comm_hunk_new_line = [comment_dict['hunk_new_start'] for comment_dict in comments]

    return pd.DataFrame(dict(
    ID=ids,
    FILE_PATH=comm_file_path,
    COMM_TYPE=comm_types,
    COMM_DATE=comm_dates,
    COMM_COMMIT_URL=comm_vcs,
    HUNK_OLD_LINE=comm_hunk_old_line,
    HUNK_NEW_LINE=comm_hunk_new_line,
    CONTENT=contents,
    PROJECT=project_names,
    ))

def get_strat_comment_df(comments: pd.DataFrame, sample_frac: float, seed: int) -> pd.DataFrame:
    shuffled_df = comments.sample(frac=1, random_state=seed)

    return shuffled_df.groupby('PROJECT', group_keys=False).apply(lambda x: stratify_prop_to_project_name(x, sample_frac))

# statistical sample size not needed as we do not use Cohens kappa on a sample of labeled data for agreement.
# check agreement with Cohens kappa and agreement level according to Fleiss (see duplicate SATD paper)
# Do need to get a statistical significant proportion of sample 

# C.I. = 5% and confidence level 95% then sample size 384..to get a statistically sign. sample for proportion p of SATD/non-SATD
# C.I. = 7% and confidence level 95% then sample size 196..to get a statistically sign. sample for proportion p of SATD/non-SATD
# C.I. = 8% and confidence level 95% then sample size 150..to get a statistically sign. sample for proportion p of SATD/non-SATD 
# C.I. = 10% and confidence level 95% then sample size 96..to get a statistically sign. sample for proportion p of SATD/non-SATD 
# start with 96 and see how it goes?
# can also compare p with proportions found in Guo et al and Maldonado datasets
stat_sign_sample_size = 95

comments_df = get_comment_df(comments)
comments_df = comments_df.drop_duplicates(subset=['PROJECT', 'CONTENT'])

# remove projects that do not have more than 2 comments
project_freq = comments_df['PROJECT'].value_counts()
projects = project_freq[project_freq>2]
comments_df = comments_df[comments_df['PROJECT'].isin(projects.index)]

random_seed_shuffle = 122324
sample_frac = stat_sign_sample_size / len(comments_df)
stratified_df = get_strat_comment_df(comments=comments_df, sample_frac=sample_frac, seed=random_seed_shuffle)



strat_project_count = len(stratified_df['PROJECT'].unique())
print(f'amount of project included in stratified sample: {strat_project_count}')

count_per_project = stratified_df.groupby(['PROJECT'])['PROJECT'].count()
print(f'amount of comments per project included in stratified sample:')
print(count_per_project)

# with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
#     display(stratified_df)

amount of project included in stratified sample: 94
amount of comments per project included in stratified sample:
PROJECT
activemq             3
airavata             3
archiva              3
avro                 3
bigtop               3
                    ..
velocity-tools       3
wss4j                3
xmlgraphics-batik    3
zeppelin             3
zookeeper            3
Name: PROJECT, Length: 94, dtype: int64


In [5]:
len(stratified_df)

282

# Persisting
---
## First Tutorial Sample

In [ ]:
tutorial_sample_size = 120
first_round_tut_sample_excel_path = Path(f'./data/first_round_tutorial_sample_no_dupl_comments_size_{tutorial_sample_size}.xlsx')
tutorial_sample_df = sample_and_persist_df(comments_df, tutorial_sample_size, first_round_tut_sample_excel_path)

with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    display(tutorial_sample_df)

## Stratified Sample

In [32]:
# shuffle so we have a fair chance of getting at least 1 sample per project labeled
# NOTE: random state was not passed initially, so we cannot possibly retrieve same sample anymore
# as we used for labeling..
sample_excel_path = Path(f'./data/no_context_no_dupl_sample_comments_set_seed_size_{len(stratified_df)}.xlsx')
stratified_df = stratified_df.sample(frac=1, random_state=2357295).reset_index(drop=True)
startified_shuff_df = sample_and_persist_df(stratified_df, 1, sample_excel_path)

# Context reference generation and persistence
---
## Second (context) Tutorial

In [9]:
# # retrieve all samples from tutorial df that are labeled as "needs context"
# # to use as tutorial sample for labeling samples with context
# satd_labeled_tutorial_sample_path = Path('./data/SATD label sheet for tutorial.xlsx')
# labeled_tut_sample_df = pd.read_excel(satd_labeled_tutorial_sample_path, sheet_name='Comments Labeled',
#                                        usecols='D',
#                                          na_values='',
#                                            nrows=len(tutorial_sample_df), ).fillna('nan')
# tutorial_sample_reset_idx_df = tutorial_sample_df.reset_index(drop=True)
# context_df = tutorial_sample_reset_idx_df[labeled_tut_sample_df.iloc[:,0].str.contains("needs context")]
# context_df.tail()

satd_labeled_tutorial_sample_path = Path('./data/SATD label sheet for tutorial.xlsx')
sec_tut_context_df = create_context_df(first_round_tut_sample_excel_path, satd_labeled_tutorial_sample_path, 'D', context_label='needs context')
sec_tut_context_df.tail()

C:\Users\twanv\AppData\Local\Temp\ipykernel_22888\2659528413.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  context_df['COMM_COMMIT_URL'] = get_commit_diff_comment_urls(


,ID,FILE_PATH,COMM_TYPE,COMM_DATE,COMM_COMMIT_URL,HUNK_OLD_LINE,HUNK_NEW_LINE,CONTENT,PROJECT
103,645f91f2a7350c5b68cd73f6,nifi-nar-bundles/nifi-scripting-bundle/nifi-sc...,GROUPED_LINE,2016-08-17 18:06:18,https://github.com/apache/nifi/blob/a5261914fb...,215,232,"If the script provides a Processor, call its ...",nifi
110,645f9211a7350c5b68ced209,src/main/java/org/apache/directory/fortress/co...,BLOCK,2016-10-09 01:58:45,https://github.com/apache/directory-fortress-c...,745,735,* Ensure the paSet is present and name is safe.,directory-fortress-core
111,645f91eca7350c5b68cd5562,nifi-nar-bundles/nifi-atlas-bundle/nifi-atlas-...,GROUPED_LINE,2017-12-18 18:23:58,https://github.com/apache/nifi/commit/fc73c609...,0,1,Only apply the function when the element is t...,nifi
115,645f910ca7350c5b68c7ab76,storage-hbase/src/main/java/org/apache/kylin/s...,GROUPED_LINE,2016-03-10 08:42:41,https://github.com/apache/kylin/blob/0ba5881aa...,0,1,create htable if it doesn't exist,kylin
117,645f9455a7350c5b68d8f5da,curator-framework/src/main/java/org/apache/cur...,GROUPED_LINE,2019-03-03 20:30:21,https://github.com/apache/curator/blob/1552755...,57,225,Just calling inflater.needsInput() doesn't wo...,curator


In [10]:
# persist context df
context_lab_tut_sample_excel_path = Path(f'./data/second_round_tutorial_sample_size_{len(sec_tut_context_df)}.xlsx')
sec_tut_context_df.to_excel(context_lab_tut_sample_excel_path, index=False)

## Context Labeling Round

In [ ]:
# just add another column to the aggregated labeling sheet where you choose a final label
# so you dont have to set all 3 labels to same thing in all 3 different sheets
# and so context generation can be same as for tutorial.

satd_labeled_first_round_sample_path = Path('insert path to first round excell file')
# TODO need to set length of the excell files being read to length of the final label column (G?)?
# should actually not be a problem.. because gets filtered on final label column anyways and non labeled
# will be filtered out?

# add context column to all 3 label sheets? just have another label sheet context and refer to this commit url column
# should be enough context? Check with tutorial.

# TODO check also the second tutorial commit urls go through all.
# they are now all filled but stil lots of blobs..

# also can use both same dataframe because we filter on one?
first_rnd_context_df = create_context_df(satd_labeled_first_round_sample_path, satd_labeled_first_round_sample_path, 'G', context_label='needs context')
first_rnd_context_df.tail()

In [ ]:
# persist context df
context_lab_first_round_sample_excel_path = Path(f'./data/first_round_context_sample_size_{len(first_rnd_context_df)}.xlsx')
first_rnd_context_df.to_excel(context_lab_first_round_sample_excel_path, index=False)

# Inter-rater Agreement
---

In [23]:
from cmath import nan
from typing import Any, Union
import krippendorff

def label_converter(label: str) -> int:
    if label == 'satd':
        return 1
    elif label == 'non-satd':
        return 0
    else:
        return nan

satd_labeled_sample_path = Path('./data/SATD data quality check.xlsx')
labeled_sample_df = pd.read_excel(satd_labeled_sample_path, sheet_name='Comments Labeled',
    usecols='D:F',
    na_values="...",
    converters={0: label_converter, 1: label_converter, 2: label_converter},
    nrows=len(stratified_df))

reliability_data = [labeled_sample_df[column].tolist() for column in labeled_sample_df]

# gives back nan if whole row of one labeler is nan
krippendorff.alpha(reliability_data=reliability_data, level_of_measurement='nominal')


0.12344113295286419

In [21]:
labeled_sample_df


,labeler 1 (Nathan),labeler 2 (Alexander),labeler 3 (Twan)
0,NaN,non-satd,non-satd
1,NaN,non-satd,non-satd
2,NaN,satd,non-satd
3,NaN,satd,non-satd
4,NaN,non-satd,non-satd
...,...,...,...
91,NaN,satd,satd
92,NaN,non-satd,non-satd
93,NaN,satd,non-satd
94,NaN,non-satd,non-satd


[WDM] - Downloading: 100%|██████████| 6.81M/6.81M [00:00<00:00, 27.9MB/s]


https://github.com/apache/commons-rdf/commit/92998b329182724789f44272f60538dc195b8df3#diff-dac69938e4093eccc15dd1907659090c2b72963a19e747a55d0f38d90e46e556R495
